<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/machine_learning/blob/main/classes/class_march_17/clase_march_17_clasificacion_multietiqueta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table> 

# Clase del 17 de Marzo. Clasificación multietiqueta 

Página 159 de Aurelien Geron

Hasta ahora, cada instancia siempre se ha asignado a una sola clase.

In some cases you may want your classifier to output multiple classes for each instance. 

Consider a face-recognition classifier: what should it do if it recognizes several people in the same picture? 

It should attach one tag per person it recognizes. 

Say the classifier has been trained to recognize three faces, Alice, Bob, and Charlie. 

Then when the classifier is shown a picture of Alice and Charlie, it should output [1, 0, 1] (meaning “Alice yes, Bob no, Charlie yes”). 

Such a classification system that outputs multiple binary tags is called a multilabel classification system.

Todavía no entraremos en el reconocimiento facial, pero veamos un ejemplo más simple, solo con fines ilustrativos:

In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [2]:
X, y = mnist["data"], mnist["target"]
X.shape

(70000, 784)

In [5]:
import numpy as np 
y = y.astype(np.int64) 

In [7]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [8]:
%%time 

from sklearn.neighbors import KNeighborsClassifier
y_train_large = (y_train >= 7)
y_train_odd = (y_train % 2 == 1)
y_multilabel = np.c_[y_train_large, y_train_odd]
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_multilabel)

Wall time: 2.4 s


KNeighborsClassifier()

This code creates a y_multilabel array containing two target labels for
each digit image: the first indicates whether or not the digit is large (7, 8, or 9), and the second indicates whether or not it is odd. 

The next lines create a KNeighborsClassifier instance (which supports multilabel classification, though not all classifiers do), and we train it using the multiple targets array.

Ahora puede hacer una predicción y notar que genera dos etiquetas:

In [9]:
knn_clf.predict([X[0]]) 

array([[False,  True]])

¡Y lo hace bien! De hecho, el dígito 5 no es grande (Falso) e impar (Verdadero).

There are many ways to evaluate a multilabel classifier, and selecting the right metric really depends on your project. 

One approach is to measure the $F_{1}$ score for each individual label (or any other binary classifier metric discussed earlier), then simply compute the average score. 

This code computes the average $F_{1}$ score across all labels:

In [ ]:
y_train_knn_pred = cross_val_predict(knn_clf, X_train, y_multilabel, cv=3)
f1_score(y_multilabel, y_train_knn_pred, average="macro")